In [1]:
import json, warnings, os
from pathlib import Path

warnings.filterwarnings('ignore')

if Path.cwd().name == "notebooks":
    rootdir = Path.cwd().parent
else:
    rootdir = Path.cwd()

import pandas as pd
import plotly.express as px
from plotly import graph_objects as go

from stables.utils.postgres import get_sqlalchemy_engine
from stables.config import local_pg_config, remote_pg_config
engine = get_sqlalchemy_engine(local_pg_config)

import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

## APY Chart

In [2]:
query = """
SELECT 
  pool_name,
  DATE(time) as day,
  tvl_usd as tvl,
  apy,
  AVG(apy) OVER (
    PARTITION BY pool_name 
    ORDER BY DATE(time) 
    ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
  ) as apy_30d_avg,
  AVG(apy) OVER (
    PARTITION BY pool_name 
    ORDER BY DATE(time) 
    ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
  ) as apy_90d_avg
FROM llama.yield_pools

"""
df = pd.read_sql(query, engine)
fig = px.line(df, x='day', y='apy', color='pool_name')
fig.show()

## Current APY

In [3]:
query = """
WITH ranked_pools AS (
  SELECT 
    pool_name,
    DATE(time) as day,
    CAST(tvl_usd/1e6 AS DECIMAL(10,2)) as tvl,
    apy,
    CAST(AVG(apy) OVER (
      PARTITION BY pool_name 
      ORDER BY DATE(time) 
      ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    ) AS DECIMAL(10,2)) as apy_30d_avg,
    CAST(AVG(apy) OVER (
      PARTITION BY pool_name 
      ORDER BY DATE(time) 
      ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
    ) AS DECIMAL(10,2)) as apy_90d_avg,
    ROW_NUMBER() OVER (
      PARTITION BY pool_name 
      ORDER BY DATE(time) DESC
    ) as rn
  FROM llama.yield_pools
)
SELECT 
  pool_name,
  -- day
  tvl,
  apy,
  apy_30d_avg,
  apy_90d_avg
FROM ranked_pools
WHERE rn = 1
ORDER BY pool_name

"""

df = pd.read_sql(query, engine)
display(df)

,pool_name,tvl,apy,apy_30d_avg,apy_90d_avg
0,fxsave,44.34,18.37000,10.51,10.19
1,scrvusd,52.52,7.17661,6.40,4.69
2,sdai,117.24,5.07724,5.50,5.51
3,sdola,27.33,9.96314,7.98,7.83
4,sfrxusd,30.68,11.77790,6.19,6.17
5,stusr,196.54,7.45427,9.10,8.02
6,susde,4079.72,10.53288,5.97,5.52
7,susds,1745.03,4.50000,4.50,4.50


# 